In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import os
# import sys
# sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json

In [ ]:
df = pd.read_csv('data/ZZP.csv', sep=";")
df['Datum'] = pd.to_datetime(df['Datum'], format='%d-%m-%Y')

In [ ]:
df.head()

In [ ]:
df['Datum'].unique()

In [ ]:
df.index = pd.DatetimeIndex(df['Datum'], freq='D')

In [ ]:
df.head()

In [ ]:
data = df['ZZP5']

In [ ]:
data

In [ ]:
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Check if the series is stationary
result = adfuller(data)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

# Plot ACF and PACF
plot_acf(data, lags=50)
plot_pacf(data, lags=50)

In [ ]:
# Fit an ARIMA model to your data
# model = ARIMA(data, order=(5,2,5), freq='D')
model = ARIMA(data, order=(1,2,5), freq='D')
model_fit = model.fit()

In [ ]:
# Forecast future data points
n_forecast = 100
forecast = model_fit.forecast(steps=n_forecast)

In [ ]:
extended_data = pd.concat([data, pd.Series(forecast)], ignore_index=True)

In [ ]:
extended_data.plot()

In [ ]:
import plotly.graph_objects as go

# Create a plot
fig = go.Figure()

# Add a line trace for the extended_data
fig.add_trace(go.Scatter(x=extended_data.index, y=extended_data, mode='lines', name='extended_data'))

# Set the title and labels
fig.update_layout(title='Extended Data Timeseries', xaxis_title='Time', yaxis_title='Value')

# Show the plot
fig.show()

In [ ]:
# Stappenplan:
# - Trendlijn bepalen
# - delta trendlijn bepalen
# - Normaaldistributie maken van delta trendlijn
# - Meerjaren trendlijn maken
# - Op basis van normaal distributie en meerjaren trendlijn voorspelling maken

In [ ]:
df = pd.read_csv('data/ZZP.csv', sep=";")
df['Datum'] = pd.to_datetime(df['Datum'], format='%d-%m-%Y')
df['ZZP_VVT'] = df['ZZP6'] + df['ZZP7'] + df['ZZP8'] + df['ZZP9'] + df['ZZP10']

data = df['ZZP_VVT']

time = np.arange(len(data))
slope, intercept = np.polyfit(time, data, 1)

extension_period = 365*3
extended_time = np.arange(len(data) + extension_period)
# Create a trend line
trend = intercept + slope * extended_time
df_extended = pd.DataFrame({'Datum': df['Datum'].iloc[0] + pd.to_timedelta(extended_time, unit='D'), 'trend': trend})

df_extended = pd.concat([df['ZZP_VVT'], df_extended], axis=1)
df_extended['delta'] = df_extended['ZZP_VVT'] - df_extended['trend']

df_extended['synthetic_delta'] = np.random.normal(loc=0.0, scale=df_extended['delta'].std(), size=len(df_extended))
df_extended['synthetic_ZZP_VVT_'] = df_extended['trend'] + df_extended['synthetic_delta']

df_extended['synthetic_ZZP_VVT'] = df_extended['synthetic_ZZP_VVT_'].rolling(window=7).mean()
df_extended = df_extended.dropna(subset=['synthetic_ZZP_VVT'])
df_extended['synthetic_ZZP_VVT'] = df_extended['synthetic_ZZP_VVT'].astype(int)

In [ ]:
ts_clients = df_extended['synthetic_ZZP_VVT'].to_dict()
dict_timeseries = {}
dict_timeseries['ts_clients'] = ts_clients

In [ ]:
# Write the updated dictionary back to a JSON file
with open('data/dict_timeseries.json', 'w') as f:
    json.dump(dict_timeseries, f)

In [ ]:
# Load the dictionary from a JSON file
with open('data/dict_timeseries.json', 'r') as f:
    dict_test = json.load(f)

In [ ]:
df_extended.to_csv('data/ZZP_extended.csv', sep=';', index=False)

In [ ]:
df['ZZP_VVT'] = df['ZZP6'] + df['ZZP7'] + df['ZZP8'] + df['ZZP9'] + df['ZZP10']



In [ ]:
data = df['ZZP_VVT']
# Assuming 'data' is your time series
time = np.arange(len(data))
slope, intercept = np.polyfit(time, data, 1)

extension_period = 365*3
extended_time = np.arange(len(data) + extension_period)
# Create a trend line
trend = intercept + slope * extended_time

In [ ]:
ts_clients = data.to_dict()

In [ ]:

# Import the dictionary from ts.py
from data.ts import dict_timeseries

# Update the dictionary
dict_timeseries['ts_clients'].update(ts_clients)

# Write the updated dictionary back to ts.py
with open('data/ts.py', 'a') as f:
    f.write(f'\dict_timeseries = {dict_timeseries}\n')

In [ ]:
import json

# Import the dictionary from ts.py
from data.ts import dict_timeseries

# Update the dictionary
dict_timeseries['ts_clients'].update(ts_clients)

# Write the updated dictionary back to a JSON file
with open('data/dict_timeseries.json', 'w') as f:
    json.dump(dict_timeseries, f)

In [ ]:
# Load the dictionary from a JSON file
with open('data/dict_timeseries.json', 'r') as f:
    dict_timeseries = json.load(f)


In [ ]:
dict_timeseries['ts_purchases']

In [ ]:
extended_time

In [ ]:
df_extended = pd.DataFrame({'Datum': df['Datum'].iloc[0] + pd.to_timedelta(extended_time, unit='D'), 'trend': trend})

In [ ]:
df_extended.head()

In [ ]:
df_extended = pd.concat([df['ZZP_VVT'], df_extended], axis=1)

In [ ]:
df_extended.shape

In [ ]:
df_extended['delta'] = df_extended['ZZP_VVT'] - df_extended['trend']

In [ ]:
df_extended = pd.concat([df['ZZP_VVT'], df_extended], axis=1)
df_extended['delta'] = df_extended['ZZP_VVT'] - df_extended['trend']

df_extended['synthetic_delta'] = np.random.normal(loc=0.0, scale=df_extended['delta'].std(), size=len(df_extended))
df_extended['synthetic_ZZP_VVT_'] = df_extended['trend'] + df_extended['synthetic_delta']

df_extended['synthetic_ZZP_VVT'] = df_extended['synthetic_ZZP_VVT_'].rolling(window=7).mean()
df_extended = df_extended.dropna(subset=['synthetic_ZZP_VVT'])
df_extended['synthetic_ZZP_VVT'] = df_extended['synthetic_ZZP_VVT'].astype(int)

In [ ]:
plt.plot(df_extended['ZZP_VVT'], label='Original')
plt.plot(df_extended['trend'], label='Trend', color='red')
plt.plot(df_extended['synthetic_ZZP_VVT'], label='Trend', color='green')
plt.legend()
plt.show()

In [ ]:


# Assuming 'data' is your time series
time = np.arange(len(data))
slope, intercept = np.polyfit(time, data, 1)

# Create a trend line
trend = intercept + slope * time

# Plot the original time series and the trend
plt.plot(data, label='Original')
plt.plot(time, trend, label='Trend', color='red')
plt.legend()
plt.show()

# Moving average
data_ma = data.rolling(window=12).mean()  # window size depends on the nature of the data

# Plot the original time series and the moving average
plt.plot(data, label='Original')
plt.plot(data_ma, label='Moving Average', color='red')
plt.legend()
plt.show()

In [ ]:
time